In [1]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_dataset = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

c:\Users\Srujan Landeri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained("test_trainer/checkpoint-1377")
tokenizer = AutoTokenizer.from_pretrained("test_trainer/checkpoint-1377")
training_args = TrainingArguments("test_trainer")

# Initialize the Trainer with the loaded model
trainer = Trainer(
    model=model,
    args=training_args,  # Use your training arguments defined earlier
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

In [3]:
# Make predictions
predicted = trainer.predict(tokenized_datasets["validation"])

# Print shapes of predictions and labels
print(predicted.predictions.shape, predicted.label_ids.shape)

# Get predicted class indices
import numpy as np
preds = np.argmax(predicted.predictions, axis=-1)

100%|██████████| 51/51 [00:17<00:00,  2.93it/s]

(408, 2) (408,)


In [4]:
import evaluate

evaluator = evaluate.load("glue", "mrpc")
results = evaluator.compute(predictions=preds, references=predicted.label_ids)
print(results)

{'accuracy': 0.8553921568627451, 'f1': 0.8970331588132635}


In [6]:
import evaluate

# defining a compute_metrics function
def compute_metrics(eval_preds):
    evaluator = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    np.argmax(logits, axis=-1)
    return evaluator.compute(predictions=np.argmax(logits, axis=-1), references=labels)

# training with additional parameters
trainer = Trainer(
    model = model,
    train_dataset= tokenized_datasets["train"],
    eval_dataset= tokenized_datasets["validation"],
    tokenizer= tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

 36%|███▋      | 500/1377 [1:40:03<20:17,  1.39s/it]      

{'loss': 0.1446, 'grad_norm': 0.026357989758253098, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [1:55:44<11:10,  1.78s/it]

{'loss': 0.0665, 'grad_norm': 0.027990322560071945, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [2:07:32<00:00,  5.56s/it]

{'train_runtime': 7652.4821, 'train_samples_per_second': 1.438, 'train_steps_per_second': 0.18, 'train_loss': 0.08438600970942124, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.08438600970942124, metrics={'train_runtime': 7652.4821, 'train_samples_per_second': 1.438, 'train_steps_per_second': 0.18, 'total_flos': 405626802939840.0, 'train_loss': 0.08438600970942124, 'epoch': 3.0})